# Exploring TSV files`

### Imports

In [ ]:
import os       # using operating system dependent functionality (folders)
import glob
import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import copy     # Can Copy and Deepcopy files so original file is untouched.
from ipywidgets import IntSlider, Output
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.insert(0, '../') # path to functions
from cvasl import file_handler as fh # 
from cvasl import mold #
from cvasl import carve
from cvasl.file_handler import Config

### Configure data

In [ ]:
config = Config()
root_mri_directory = config.get_directory('raw_data')

### Load tsv files

In [ ]:
tsv_pattern = os.path.join(root_mri_directory, '**/*.tsv')
tsv_files = glob.glob(tsv_pattern, recursive=True)


### check tsv files

In [ ]:
dataframe_example = pd.read_csv(tsv_files[0], sep='\t')

In [ ]:
dataframe_example

In [ ]:
dataframe_example.columns

### Analysis

Without the subject ages we can not do an analysis on anything except how subjects progress over time points, and how various parameters predict each other. But let's scan all the tev we were given to see if we have ones with age. 

#### Correlations within datasets:

Let's use one dataframe example:

In [ ]:
dataframe_example.columns

In [ ]:
dataframe_example_nums_only = dataframe_example[[ 
    'SubjectNList',
    'Site', 'GM_vol', 'WM_vol', 'CSF_vol', 'GM_ICVRatio', 'GMWM_ICVRatio',
    'WMH_vol', 'WMH_count', 'MeanMotion', 'DeepWM_B', 'DeepWM_L',
    'DeepWM_R']]

In [ ]:
dataframe_example_nums_only

In [ ]:
dataframe_example_nums_only[1:].corr()

In [ ]:


%matplotlib inline
sns.heatmap(dataframe_example_nums_only[1:].corr(), annot = True)



So we do see some correlations int his particular dataset that are strong an expected. The area of brain to ICV ratio negatively correlates with the CSF volume as should be expected. White matter and grey matter correlate pretty well. White matter hyperintensities in count correlate somewhat with white matter hyperintensity volume. And deep WM-L correlates between the sides of the brain and both. Basically everything we would expect. 

So now we can make a super-dataset of all the datasets, and see if these correlations hold. 

In [ ]:
dataframe_example2 = pd.read_csv(tsv_files[1], sep='\t')
dataframe_example2.columns.to_list()

More elements than first...let's see what we have in common between the two tsv:

In [ ]:
first = set(dataframe_example.columns.to_list())
second = set(dataframe_example2.columns.to_list())

In [ ]:
not_common2 =  list(set(dataframe_example2.columns.to_list()) - set(dataframe_example.columns.to_list()))

In [ ]:
shared = list(first.intersection(second))
shared

Not a lot...let's look at what we have in common in all or most of the tsv

In [ ]:
name_file = []
longlesses = []
for file in tsv_files:
    dataframe_example = pd.read_csv(file, sep='\t')
    longness = len(dataframe_example.columns)
    name_file.append(file)
    longlesses.append(longness)
data_tsv = pd.DataFrame([name_file, longlesses])        
print(longlesses)        

In [ ]:
data_tsv

So There may be 15 common features on most as a guess. We need to not look at the last on the list.

In [ ]:
#print(name_file[:-1])

In [ ]:
set_of_relevant_files = name_file[:-1]

In [ ]:
name_file = []
longlesses = []
intersections = []
len_intersections = []
for file in set_of_relevant_files:
    dataframe_example = pd.read_csv(file, sep='\t')
    longness = len(dataframe_example.columns)
    name_file.append(file)
    longlesses.append(longness)
    dataframe_example2 = pd.read_csv(file, sep='\t')
    columns = dataframe_example2.columns.to_list()
    intersection = set(columns).intersection(second)
    intersections.append(intersection)
    len_intersections.append(len(intersection))
data_tsv = pd.DataFrame([name_file, longlesses, len_intersections, intersections])         

In [ ]:
data_tsv

So we will have twelve or thireen common elements we can compare.Let's look at hope they are about the same.

In [ ]:
data_tsv[0][3]

In [ ]:
data_tsv[2][3]

In [ ]:
data_tsv[4][3]

In [ ]:
data_tsv[4][3].intersection(data_tsv[2][3]).intersection(data_tsv[0][3]).intersection(data_tsv[5][3]).intersection(data_tsv[3][3]).intersection(data_tsv[6][3]).intersection(data_tsv[7][3])

OK, so more or less we should have the above values in every group of tsv in our supergroup.
Let's check if we do

In [ ]:
list_elements = data_tsv[4][3].intersection(data_tsv[2][3]).intersection(data_tsv[0][3]).intersection(data_tsv[5][3]).intersection(data_tsv[3][3]).intersection(data_tsv[6][3]).intersection(data_tsv[7][3])

In [ ]:
list_elements

In [ ]:
name_file = []
longlesses = []
good_files = []
for file in tsv_files:
    dataframe_example = pd.read_csv(file, sep='\t')
    if set(list_elements).issubset(set(dataframe_example.columns.to_list())):
                                   good_files.append(file)
       
print(good_files)        

Here we must say in this set, the set of common elements is about:
CSV_vol,  'GMWM_ICVRatio',
 'GM_ICVRatio',
 'GM_vol',
 'LongitudinalTimePoint',
 'MeanMotion',
 'Site',
 'SubjectNList',
 'WMH_count',
 'WMH_vol',
 'WM_vol',
 'participant_id',
 'session'
 
 However we need something to extract the common set from any group of tsv columns

In [ ]:
def make_columns(list_tsv_files):
    columns_list = []
    for file in list_tsv_files:
        dataframe_example = pd.read_csv(file, sep='\t')
        columns= dataframe_example.columns.to_list()
        columns_list.append(columns)
    return columns_list

a = fh.make_columns(tsv_files[:8])



In [ ]:

def intersect_all(*sets):
    result, *rest = sets
    for remaining in rest:
        result = set(result).intersection(remaining)
    return result

good_columns_sets = fh.intersect_all(*a)

Now we make our super tsv file

In [ ]:
good_columns_list = list(good_columns_sets)
good_columns_list

In [ ]:
tabs_together = []
for file in tsv_files[:8]:
    print(file)
    tabular = pd.read_csv(file, sep='\t')
    tabularnow = tabular[good_columns_list]
    tabs_together.append(tabularnow)

In [ ]:
tabs_together

now if tsvs were different we could stack 8 elements of tabular ...and make a supercomparator, but we seem tohave the same tsv over and over...wierd check wth scientists